In [ ]:
### Define a station

import pytz

station = 'KBGM'

# station info (from https://weather.gladstonefamily.net/site/KBGM)
station_lat =  42.2068
station_lon = -75.9799
station_elev = 499 # meters
station_tz_name = "US/Eastern"
station_tz  = pytz.timezone(station_tz_name)

In [ ]:
if start_date.year == end_date.year:
    if start_date.month == end_date.month and start_date.day == end_date.day:
        date_range = ""
        plot_title_suffix = start_date.strftime("%b %-d, %Y")
    elif start_date.month == end_date.month:
        start_date_print = start_date.strftime("%b %-d")
        end_date_print = end_date.strftime("%-d")
        date_range = f"{start_date_print}-{end_date_print}"

        plot_title_suffix = start_date.strftime(", %Y")
    else: # months differ, years same
        start_date_print = start_date.strftime("%b %-d")
        end_date_print = end_date.strftime("%b %-d")
        date_range = f"{start_date_print} to {end_date_print}"

        plot_title_suffix = start_date.strftime(", %Y")
else: # years differ
    start_date_print = start_date.strftime("%-d/%-m/%Y")
    end_date_print = end_date.strftime("%-d/%-m/%Y")
    date_range = f"{start_date_print} to {end_date_print}"

    plot_title_suffix = ""

plot_title = f"Vertical profiles of refl @{station}, {date_range}{plot_title_suffix}"
# print(plot_title)

start_date_filename = start_date.strftime("%Y_%m_%d")
end_date_filename = end_date.strftime("%Y_%m_%d")
plot_filename = f"vpr_heatmap_{station}_{start_date_filename}-{end_date_filename}"
# print(plot_filename)

In [ ]:
import glob

root = '.'

# the VPRs are sorted by day and in UTC time, so construct a list of all UTC days between start and end
daylist = pd.date_range(start_date.astimezone(pytz.utc), end_date.astimezone(pytz.utc), normalize=True, freq='D')

# list all VPR files that could be loaded
scan_files = []
for day in daylist:
#     print(day + pd.DateOffset(1))
#     print(f"{root}/{day.year}/{day.month:02d}/{day.day:02d}/{station}/*.csv")
    scan_files = scan_files + glob.glob(f"{root}/{day.year}/{day.month:02d}/{day.day:02d}/{station}/*.csv")
    
scan_files = sorted(scan_files)

In [ ]:
import pandas as pd
import numpy as np
import os

# read in all the scans

n_height_bins = 30
height_bins = np.arange(0,n_height_bins) * 100
all_scans = pd.DataFrame(columns=height_bins)

for s, scan_file in enumerate(scan_files):
    scan_key = os.path.splitext(os.path.basename(scan_file))[0]
    
    # read in the profile data    
    scan_data = pd.read_csv(scan_file,
                            usecols = ['bin_lower', 'linear_eta'],
                            index_col = 'bin_lower').T
    
    scan_data.rename(index={'linear_eta': scan_key}, inplace=True)
    
    all_scans = all_scans.append(scan_data)
    
all_scans.rename_axis(columns='', inplace=True)

n_scans = len(all_scans.index)

In [ ]:
# from datetime import datetime

# parse the scan_key into a datetime object
UTC_time_indices = pd.Series()
local_time_indices = pd.Series()
for scan_key, row in all_scans.iterrows():
    # parse the scan key
    station = scan_key[0:4]
    year = int(scan_key[4:8])
    month = int(scan_key[8:10])
    day = int(scan_key[10:12])
    hour = int(scan_key[13:15])
    minute = int(scan_key[15:17])
    second = int(scan_key[17:19])
    
    t_UTC = pd.Timestamp(year=year, month=month, day=day, 
                     hour=hour, minute=minute, second=second,
                     tz=pytz.utc)
    t_local = t_UTC.astimezone(station_tz)
    
    UTC_time_indices[scan_key] = t_UTC
    local_time_indices[scan_key] = t_local
    
    
# set the datetime object as the index (and cache the scan_key as a column)
all_scans_time = all_scans.copy()
all_scans_time.insert(0, 'scan_key', all_scans_time.index)
all_scans_time['t_UTC'] = UTC_time_indices
all_scans_time['t_local'] = local_time_indices
all_scans_time.set_index('t_UTC', inplace=True)

print(all_scans_time)

In [ ]:
# trim scans in all_scans_time outside [start_date, end_date]
n_scans_raw = n_scans

mask = (all_scans_time['t_local'] >= start_date) & (all_scans_time['t_local'] <= end_date)
all_scans_time = all_scans_time.loc[mask]

# recompute the number of scans remaining after trimming
n_scans = len(all_scans_time.index)

print(f"{n_scans}/{n_scans_raw} loaded scans remaining.")

In [ ]:
import astral

# compute the solar elevation angle (deg above the horizon) for each scan

a = astral.Astral()

solar_elevs = pd.Series()
for scan_dt, row in all_scans_time.iterrows():
    solar_elevs[scan_dt] = a.solar_elevation(scan_dt, station_lat, station_lon)

all_scans_time.loc[:,'solar_elev'] = solar_elevs

# print(all_scans_time)

In [ ]:
import matplotlib.colors as colors

# a colormap normalizer centered at civil twilight (-6deg below horizon)
class CivilTwilightNorm(colors.Normalize):
    def __init__(self, vmin=-90, vmax=90):
        self.midpoint = -6
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.interp(value, x, y)

# initialize a CTN w/ extent defined by the values in all_scans
all_scans_norm = CivilTwilightNorm(np.min(all_scans_time['solar_elev']), np.max(all_scans_time['solar_elev']))
    
# plt.imshow(np.array(all_scans['solar_elev']).reshape(1,-1),
#            norm=CivilTwilightNorm(np.min(all_scans['solar_elev']), np.max(all_scans['solar_elev'])),
#            cmap='bone')
# plt.axis('off')

In [ ]:
# compute the ticks and tick labels

x_tick_freq = 15
xticks = np.arange(0, n_scans - 1, x_tick_freq)
xticklabels = all_scans_time.iloc[xticks]['t_local']
# print(xticklabels)

y_tick_freq = 5
yticks = np.arange(0, n_height_bins, y_tick_freq)
yticklabels = yticks * 100
# print(yticklabels)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

%matplotlib inline

fig_format = '.pdf' # vect. text in pdf is much better, but the pixels of the heatmap tend to have a small white grid
                    # on my Mac, the only tool that doesn't get a grid is Adobe Acrobat Reader
# fig_format = '.png' # text in png is bad, but there's no grid between heatmap pixels

fig_height = 5 # inches
fig_min_width = 2 # inches
fig_width_per_scan = 0.05 # inches

highlight_over = True

# the daylight bar is overlaid just above the heatmap
# this grid aligns the daylight bar and heatmap, with the heatmap colorbar on the side
nrow = 50 # increase to shrink the daylight bar
ncol = int(n_scans / 10) # increase to shrink the colorbar
grid = plt.GridSpec(nrow,ncol,hspace = 0)
fig = plt.figure(figsize=[fig_min_width + n_scans * fig_width_per_scan, fig_height])

# build the grid (the daylight bar is only one row, and the colorbar only one col)
ax_daylight = plt.subplot(grid[0,0:ncol-1])
ax_heatmap = plt.subplot(grid[1:nrow-1,0:ncol-1])
ax_cbar = plt.subplot(grid[:,ncol-1])

# draw the daylight bar
ax_daylight.imshow(np.array(all_scans_time['solar_elev']).reshape(1,-1),
                   norm=all_scans_norm,
                   cmap='bone')
ax_daylight.axis('off')

# draw the heatmap
heatmap_cm = cm.get_cmap('pink')
if highlight_over:
    heatmap_cm.set_over("red") # highlight values above the colormap max

all_refl = all_scans_time[height_bins].values.flatten()
vmax = np.percentile(all_refl, 99.99)
sns.heatmap(all_scans_time[height_bins].T,
            cmap = heatmap_cm, vmin = 0, vmax = vmax,
            ax = ax_heatmap, cbar_ax=ax_cbar,
            xticklabels=xticklabels, yticklabels=yticklabels)
ax_heatmap.invert_yaxis()
ax_heatmap.set_xticks(xticks);
ax_heatmap.set_yticks(yticks);

ax_heatmap.set_ylabel("height above ground (m)");
ax_heatmap.set_xlabel(f"local time ({station_tz_name})");

ax_cbar.set_ylabel("total reflectivity (dbZ)");

fig.suptitle(plot_title);

fig.savefig(f"{plot_filename}{fig_format}", bbox_inches='tight');

In [ ]:
pd.date_range(start=start_date, end=end_date, freq='D')

In [ ]:
pd.DateOffset(days='D')